## importing neccasry library 

In [39]:
from nsepy import get_history
import numpy as np
import pandas as pd 
from datetime import date 
import matplotlib.pyplot as plt 
%matplotlib inline 
from statsmodels.tsa.stattools import OLS 
from statsmodels.tsa.stattools import adfuller

##  craeting sector list with thier respective sectors stocks selection of stocks was made based on volume 

In [40]:
# craeter sector wise variables with stocks list in it 
banking = ['ALBK','ANDHRABANK','AXISBANK','BANKBARODA','BANKINDIA','CANBK','FEDERALBNK','HDFC','ICICIBANK','IDBI','INDIANB','KOTAKBANK','ORIENTBANK','PNB','SBIN','SYNDIBANK','UNIONBANK']
it = ['INFY',"TCS","WIPRO","HCLTECH","TECHM","HEXAWARE"]
cement = ["INDIACEM","ULTRACEMCO","AMBUJACEM","ACC","CENTURYTEX"]
petroleum = ["RIIL","ONGC","GAIL","IOC","BPCL","HINDPETRO","PETRONET"]
auto = ["ESCORTS","M&M","TATAMOTORS","ASHOKLEY","BAJAJ-AUTO","HEROMOTOCO","TVSMOTOR"]
health = ["AUROPHARMA","SUNPHARMA","WOCKPHARMA","CADILAHC","PEL","BIOCON","LUPIN","GLENMARK","CIPLA"]
finance = ["RELCAPITAL","DHFL","IBULHSGFIN","MANAPPURAM","L&TFH","IFCI","RECLTD","PFC","HDFC"]

In [35]:
all_sectors_stocks = [banking,it,cement,petroleum,auto,health,finance]# storing all sectors in one list 

## craeting a user defined function wich calauclates rollign adf ,rolling beta,and  mean of that rolling adf 

In [41]:
def rolling_adf(x_name,y_name,start_date,end_date):
    rolling_beta = []
    adf_values = []
    x = get_history(x_name,start = start_date,end=end_date)['Close']
    y = get_history(y_name,start = start_date,end=end_date)['Close']
    day_count = 0
    iteration_count = -1
    for i in x:                     #x is just birng used as a range
        day_count = day_count + 1    # day count turn to one in the first iteration and so on 
        if day_count >= 60:
            iteration_count = iteration_count + 1 #iteration starts adding up once day count turn 60 
            model = OLS(y[iteration_count:day_count].values,x[iteration_count:day_count].values).fit()
            spread = y[iteration_count:day_count] - (x[iteration_count:day_count] * model.params[0])
            adf = adfuller(spread)[0]
            rolling_beta.append(model.params[0])
            adf_values.append(adf)
    rolling_adf_mean = np.mean(adf_values)
    rolling_values = [adf_values,rolling_beta,rolling_adf_mean]
    return rolling_values

In [42]:
start_date = date(2016,6,1)
end_date = date(2016,12,30)

In [46]:
def dataframe(sector_dict,dict_pairs):
    rolling_adf_pvalue_mean = []
    rolling_beta_value = []
    x_stocks = []
    y_stocks = []
    for x,y in dict_pairs:
        adf_mean = sector_dict[str(x) + str(y)][2]
        rolling_beta_mean = np.mean(sector_dict[str(x) + str(y)][1])
        rolling_adf_pvalue_mean.append(adf_mean)
        x_stocks.append(x)
        y_stocks.append(y)
        rolling_beta_value.append(rolling_beta_mean)
    
    df = pd.DataFrame({"x_stock":x_stocks,"y_stock":y_stocks,
                                 "rolling_adf_mean":rolling_adf_pvalue_mean,
                                 "rolling_beta_mean":rolling_beta_value})
    return df


## banking sector test for year 2016

In [43]:
banking_pairs= [(x,y) for x in banking for y in banking if x!=y] #craeting combination of stocks so the are iterable 
banking_dict = {}
for x,y in banking_pairs:
    banking_dict[str(x) + str(y)] = rolling_adf(x,y,start_date,end_date)

In [44]:
banking_tradable_pairs = []
banking_tradable_stocks = []
for x,y in banking_pairs:
    mean = banking_dict[str(x) + str(y)][2]
    if mean <= -2.5:
        banking_tradable_pairs.append((x,y))
        banking_tradable_stocks.append(x)
        banking_tradable_stocks.append(y)
banking_tradable_pairs = set(tuple(sorted(p)) for p in banking_tradable_pairs)# removing the duplicates

In [45]:
print(set(banking_tradable_stocks))# i need this so i could copy this to my blue shift algo 
print(banking_tradable_pairs)

{'ANDHRABANK', 'ALBK', 'KOTAKBANK', 'ICICIBANK', 'BANKINDIA', 'UNIONBANK', 'SYNDIBANK', 'HDFC', 'IDBI', 'AXISBANK', 'BANKBARODA'}
{('BANKBARODA', 'UNIONBANK'), ('HDFC', 'SYNDIBANK'), ('ICICIBANK', 'IDBI'), ('BANKBARODA', 'KOTAKBANK'), ('BANKBARODA', 'HDFC'), ('BANKINDIA', 'HDFC'), ('ANDHRABANK', 'HDFC'), ('IDBI', 'KOTAKBANK'), ('ANDHRABANK', 'SYNDIBANK'), ('ALBK', 'HDFC'), ('KOTAKBANK', 'SYNDIBANK'), ('BANKBARODA', 'BANKINDIA'), ('ALBK', 'AXISBANK'), ('BANKINDIA', 'KOTAKBANK')}


In [47]:
dataframe(banking_dict,banking_tradable_pairs)

,x_stock,y_stock,rolling_adf_mean,rolling_beta_mean
0,BANKBARODA,UNIONBANK,-2.793855,0.868929
1,HDFC,SYNDIBANK,-2.682138,0.054368
2,ICICIBANK,IDBI,-2.728520,0.274727
3,BANKBARODA,KOTAKBANK,-2.546249,4.864839
4,BANKBARODA,HDFC,-2.532600,8.456695
5,BANKINDIA,HDFC,-3.117497,11.922738
6,ANDHRABANK,HDFC,-2.643492,23.565684
7,IDBI,KOTAKBANK,-2.612306,10.878303
8,ANDHRABANK,SYNDIBANK,-2.963011,1.282287
9,ALBK,HDFC,-2.752355,17.706575


## it sector test 2016

In [ ]:
it_pairs= [(x,y) for x in it for y in it if x!=y] #craeting combination of stocks so the are iterable 
it_dict = {}

In [ ]:
for x,y in it_pairs:
    it_dict[str(x) + str(y)] = rolling_adf(x,y,start_date,end_date)

In [ ]:
it_tradable_pairs = []
it_tradable_stocks = []
for x,y in it_pairs:
    mean = it_dict[str(x) + str(y)][2]
    if mean <= -2.5:
        it_tradable_pairs.append((x,y))
        it_tradable_stocks.append(x)
        it_tradable_stocks.append(y)
it_tradable_pairs = set(tuple(sorted(p)) for p in it_tradable_pairs)

In [ ]:
print(set(it_tradable_stocks))
print(it_tradable_pairs)

## cement sector test 2016

In [ ]:
cement_pairs= [(x,y) for x in cement for y in cement if x!=y] #craeting combination of stocks so the are iterable 
cement_dict = {}

In [ ]:
for x,y in cement_pairs:
    cement_dict[str(x) + str(y)] = rolling_adf(x,y,start_date,end_date)

In [ ]:
cement_tradable_pairs = []
cement_tradable_stocks = []
for x,y in cement_pairs:
    mean = cement_dict[str(x) + str(y)][2]
    if mean <= -2.5:
        cement_tradable_pairs.append((x,y))
        cement_tradable_stocks.append(x)
        cement_tradable_stocks.append(y)
cement_tradable_pairs = set(tuple(sorted(p)) for p in cement_tradable_pairs)

In [ ]:
print(set(cement_tradable_stocks))
print(cement_tradable_pairs)

In [48]:
dataframe(it_dict,it_tradable_pairs)

,x_stock,y_stock,rolling_adf_mean,rolling_beta_mean


##  petroleum sector test 2016

In [ ]:
petroleum_pairs= [(x,y) for x in petroleum for y in petroleum if x!=y] #craeting combination of stocks so the are iterable 
petroleum_dict = {}

In [ ]:
for x,y in petroleum_pairs:
    petroleum_dict[str(x) + str(y)] = rolling_adf(x,y,start_date,end_date)

In [ ]:
petroleum_tradable_pairs = []
petroleum_tradable_stocks = []
for x,y in petroleum_pairs:
    mean = petroleum_dict[str(x) + str(y)][2]
    if mean <= -2.5:
        petroleum_tradable_pairs.append((x,y))
        petroleum_tradable_stocks.append(x)
        petroleum_tradable_stocks.append(y)
petroleum_tradable_pairs = set(tuple(sorted(p)) for p in petroleum_tradable_pairs)

In [ ]:
print(set(petroleum_tradable_stocks))
print(petroleum_tradable_pairs)

## auto sector test 2016

In [ ]:
auto_pairs= [(x,y) for x in auto for y in auto if x!=y] #craeting combination of stocks so the are iterable 
auto_dict = {}

In [ ]:
for x,y in auto_pairs:
    auto_dict[str(x) + str(y)] = rolling_adf(x,y,start_date,end_date)

In [ ]:
auto_tradable_pairs = []
auto_tradable_stocks = []
for x,y in auto_pairs:
    mean = auto_dict[str(x) + str(y)][2]
    if mean <= -2.5:
        auto_tradable_pairs.append((x,y))
        auto_tradable_stocks.append(x)
        auto_tradable_stocks.append(y)
auto_tradable_pairs = set(tuple(sorted(p)) for p in auto_tradable_pairs)

In [ ]:
print(set(auto_tradable_stocks))
print(auto_tradable_pairs)

## health sector test 2016

In [ ]:
health_pairs= [(x,y) for x in health for y in health if x!=y] #craeting combination of stocks so the are iterable 
health_dict = {}

In [ ]:
for x,y in health_pairs:
    health_dict[str(x) + str(y)] = rolling_adf(x,y,start_date,end_date)

In [ ]:
health_tradable_pairs = []
health_tradable_stocks = []
for x,y in health_pairs:
    mean = health_dict[str(x) + str(y)][2]
    if mean <= -2.5:
        health_tradable_pairs.append((x,y))
        health_tradable_stocks.append(x)
        health_tradable_stocks.append(y)
health_tradable_pairs = set(tuple(sorted(p)) for p in health_tradable_pairs)

In [ ]:
print(set(health_tradable_stocks))
print(health_tradable_pairs)

## finance sector test for year 2016

In [ ]:
finance_pairs= [(x,y) for x in finance for y in finance if x!=y] #craeting combination of pairs so the are iterable 
finance_dict = {}

In [ ]:
for x,y in finance_pairs:
    finance_dict[str(x) + str(y)] = rolling_adf(x,y,start_date,end_date)

In [ ]:
finance_tradable_pairs = []
finance_tradable_stocks = []
for x,y in finance_pairs:
    mean = finance_dict[str(x) + str(y)][2]
    if mean <= -2.5:
        finance_tradable_pairs.append((x,y))
        finance_tradable_stocks.append(x)
        finance_tradable_stocks.append(y)
finance_tradable_pairs = set(tuple(sorted(p)) for p in finance_tradable_pairs)
        


In [ ]:
print(set(finance_tradable_stocks))
print(finance_tradable_pairs)